### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ

#### Факультет физико-математических и естественных наук  
#### Кафедра математического моделирования и искусственного интеллекта 

## ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ № 2

#### Дисциплина: Интеллектуальный анализ данных

##### Студент: Матюхин Григорий Васильевич
##### Группа: НПИбд-01-21

## Москва 2024
***

### Вариант №11

- Алгоритм: FPMax 
- День недели (поле order_dow таблицы orders): “4” 
- Код департамента (поле department_id таблицы products): “5” 
- Запрос: определить час дня, в который было совершено более всего покупок (заказов) 
- Показатель оценки ассоциативных правил: рычаг (leverage) 

# Выполнение

In [1]:
TARGET_ORDER_DOW = 4
TARGET_DEPARTMENT_ID = 5

## 1. Открыть базу данных

In [2]:
import sqlite3
db = sqlite3.connect("instacart.db")

In [3]:
c = db.cursor()
c.execute("SELECT name FROM sqlite_master")
c.fetchall()

[('aisles',),
 ('products',),
 ('departments',),
 ('orders',),
 ('order_products__train',)]

## 2. Загрузить данные в датафрейм

In [4]:
import pandas

In [5]:
dpts = pandas.read_sql("SELECT * FROM departments", db)
dpts.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [6]:
dpts.set_index("department_id", inplace=True)

In [7]:
prod = pandas.read_sql("SELECT * FROM products", db)
prod.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [8]:
prod.set_index("product_id", inplace=True)

In [9]:
ords = pandas.read_sql("SELECT * FROM orders", db)
ords.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,08,
1,2398795,1,prior,2,3,07,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,07,29.0
4,431534,1,prior,5,4,15,28.0


In [10]:
ords.set_index("order_id", inplace=True)

In [11]:
orders = pandas.read_sql(
    "SELECT op.*, o.order_hour_of_day FROM order_products__train op JOIN orders o ON op.order_id = o.order_id JOIN products p ON op.product_id = p.product_id WHERE o.order_dow=? AND p.department_id=?",
    db,
    params=(TARGET_ORDER_DOW, TARGET_DEPARTMENT_ID))
orders.head()

,order_id,product_id,add_to_cart_order,reordered,order_hour_of_day
0,877974,1808,1,0,11
1,1859940,15511,1,1,17
2,3409264,2120,3,1,08
3,1881604,10607,6,1,12
4,1881604,29509,2,1,12


In [12]:
print("Total rows:", len(orders))
print("Transactions:", len(orders.order_id.unique()))
print("Products:", len(orders.product_id.unique()))

Total rows: 958
Transactions: 455
Products: 314


## 3. Определить час дня, в который было совершено более всего покупок (заказов) 

In [13]:
orders.groupby("order_hour_of_day").size().idxmax()

'15'

## 4. Транзакционная база данных

In [14]:
xacts = (orders
        .merge(prod, left_on='product_id', right_index=True)
        .drop('product_id', axis=1)
        .groupby('order_id')['product_name']
        .apply(list)
    )
xacts

order_id
1007120    [Prosecco, 12 Oz Lager, Ksa Ko?Lsch Style Ale,...
1007997    [Pinot Noir, Sauvignon Blanc, Cabernet Sauvign...
1009684                                    [Pinot Noir Wine]
1009730                                     [Tequila Silver]
1014150                                    [Sauvignon Blanc]
                                 ...                        
972664     [Woodbridge Cabernet Sauvignon, 80  Vodka Holi...
979819                         [Pinot Noir, Chardonnay Wine]
983217                                  [Cabernet Sauvignon]
990839                              [London Dry Gin England]
997158                                    [Ultra Light Beer]
Name: product_name, Length: 455, dtype: object

In [15]:
max_items_xact = xacts.apply(len).idxmax()
max_items_xact

'2253479'

In [16]:
list(xacts[max_items_xact])

['Westfalia Red Ale',
 "Little Sumpin' Sumpin' Ale",
 'Belgium Beer',
 'Mighty Dry Hard Cider',
 'Belgian White Wheat Ale',
 'Crisp Hard Cider Crisp Apple',
 'Cabernet Sauvignon',
 'Scrimshaw Pilsner Style Beer',
 'Merlot',
 'Day Time Fractional IPA',
 '90 Minute Imperial Ipa',
 'Red Wine, Dark, California, 2013',
 'Villager Ipa']

## 5. Бинарная транзакционная база данных

In [17]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_fit = te.fit(xacts).transform(xacts)
te_fit

array([[ True,  True, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [18]:
binary_df = pandas.DataFrame(te_fit, columns=te.columns_)
binary_df

,12 Oz Beer,12 Oz Lager,12 Year Old Single Malt Scotch Speyside,1664,312 Urban Wheat,312 Urban Wheat Ale,46 / 94 Proof Bourbon Kentucky Whiskey,60 Minute IPA,80 Vodka Holiday Edition,805,...,Viognier,Vodka,Vodka 525,Washington State Merlot,Westfalia Red Ale,Whiskey,White Label Blended Scotch Whisky,"White Wine Winemaker's Blend California, 2010",Woodbridge Cabernet Sauvignon,Zinfandel
0,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
451,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
452,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
453,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [19]:
product_counts = binary_df.sum().sort_values(ascending=False)
top_products = product_counts.head(3)
print(f"Top products: \n{top_products}")

Top products: 
Beer                  45
Cabernet Sauvignon    43
Sauvignon Blanc       38
dtype: int64


## 6. Поиск популярных наборов

In [20]:
from mlxtend.frequent_patterns import fpmax

absolute_min_support = 3
relative_min_support = absolute_min_support / len(binary_df)

frequent_itemsets = fpmax(binary_df, min_support=relative_min_support, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.006593,(12 Oz Lager)
1,0.006593,(312 Urban Wheat Ale)
2,0.006593,(90 Minute Imperial Ipa)
3,0.006593,"(Ale, Amber)"
4,0.006593,(Alto Adige Pinot Grigio)


In [21]:
max_len = frequent_itemsets.itemsets.apply(len).max()
max_len

np.int64(2)

In [22]:
frequent_itemsets[frequent_itemsets.itemsets.apply(lambda x: len(x) == max_len)]

,support,itemsets
9,0.006593,"(Cabernet Sauvignon, Cabernet Sauvignon Sonoma..."
47,0.006593,"(Beer, Mixed 12 Pack Lion's Share Ale)"
52,0.006593,"(Tequila Silver, Chardonnay)"
55,0.006593,"(Little Sumpin' Sumpin' Ale, Longboard Island ..."
57,0.006593,"(IPA, Crisp Hard Cider Crisp Apple)"
60,0.006593,"(Frontier Whiskey, India Pale Ale)"
61,0.008791,"(Cabernet Sauvignon, Merlot)"
62,0.006593,"(Sauvignon Blanc, California Red Wine)"
64,0.006593,"(India Pale Ale, Premium Belgian Lager)"
67,0.006593,"(Cabernet Sauvignon, Variety Pack Hard Cider)"


## 7. Ассоциативные правила

Использование `fpmax` на предудущем шаге, а также выбор только элементов длинной два ломают постороение ассоциативных правил.
Такое поведение `fpmax` уже было замечено, но не исправлено https://github.com/rasbt/mlxtend/issues/1030, поэтому создадим наши популярные наборы заново используя `fpgrowth`.

Также в задании не задан порог уверенности, поэтому выберем его случайно.

In [23]:
import random
min_conf = random.random()
min_conf

0.206053025202655

In [24]:
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

frequent_itemsets = fpgrowth(binary_df, min_support=relative_min_support, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_conf)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Pinot Noir),(Cabernet Sauvignon),0.048352,0.094505,0.013187,0.272727,2.885835,0.008617,1.245055,0.686682
1,(Pinot Noir Wine),(India Pale Ale),0.021978,0.068132,0.006593,0.300000,4.403226,0.005096,1.331240,0.790262
2,(Pinot Noir Wine),(Cabernet Sauvignon),0.021978,0.094505,0.006593,0.300000,3.174419,0.004516,1.293564,0.700375
3,(Tequila Silver),(Chardonnay),0.010989,0.079121,0.006593,0.600000,7.583333,0.005724,2.302198,0.877778
4,(Belgian White Wheat Ale),(Beer),0.024176,0.098901,0.010989,0.454545,4.595960,0.008598,1.652015,0.801802
5,(Belgian White Wheat Ale),(Cabernet Sauvignon),0.024176,0.094505,0.006593,0.272727,2.885835,0.004309,1.245055,0.669670
6,(Little Sumpin' Sumpin' Ale),(India Pale Ale),0.021978,0.068132,0.006593,0.300000,4.403226,0.005096,1.331240,0.790262
7,(Little Sumpin' Sumpin' Ale),(Cabernet Sauvignon),0.021978,0.094505,0.008791,0.400000,4.232558,0.006714,1.509158,0.780899
8,(Chardonnay),(Cabernet Sauvignon),0.079121,0.094505,0.017582,0.222222,2.351421,0.010105,1.164207,0.624105
9,(Extra Beer Bottles),(India Pale Ale),0.019780,0.068132,0.008791,0.444444,6.523297,0.007444,1.677363,0.863789


## 8. Поиск лучшего ассоциативного правила по заданной метрике

In [25]:
rules.sort_values("leverage", ascending=False).head(1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
12,(Beer),(India Pale Ale),0.098901,0.068132,0.021978,0.222222,3.261649,0.01524,1.198116,0.769512
